In [1]:
import sys
!{sys.executable} -m pip install bs4

In [2]:
!{sys.executable} -m pip install lxml

In [30]:
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 6.0MB/s ta 0:00:011
     |████████████████████████████████| 829kB 22.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [79]:
# import some libs
from bs4 import BeautifulSoup
import pandas as pd
import requests
import lxml
import folium
from sklearn.cluster import KMeans
import numpy as np

In [4]:
#beautiful soupy soup doin some work
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html')
tbl = soup.find("table")

In [6]:
#pandas needs some data
df = pd.read_html(tbl.prettify(), flavor = 'bs4')

In [11]:
#now we have the data. Since read_html returns a list we'll just get the first table from the list of one table. Yeah.
df_Toronto = df[0]
df_Toronto

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [21]:
df_filter = df_Toronto[df_Toronto['Borough'] != 'Not assigned']

In [46]:
df_filter.reset_index().drop(['index'], axis = 1)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [57]:
# import geocoder # import geocoder
# coord_list = []

# for i in range((len(df_filter.index))):
#     # initialize your variable to None
#     lat_lng_coords = None
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(str(df_filter.iloc[i, 0])))
#         lat_lng_coords = g.latlng
#         print(lat_lng_coords)

#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#     coord_list.append((latitude,longitude)) 

In [54]:
df_loc = pd.read_csv('Geospatial_Coordinates.csv')

In [55]:
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [64]:
df_new = pd.merge(df_filter, df_loc, how = 'left')
df_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [73]:
m = folium.Map(location=[43.6532, -79.3832], zoom_start= 12)
m

In [77]:
for i in range(len(df_new.index)):
    folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='red')).add_to(m)

In [78]:
m

In [86]:
# K Means implementation. You know a manhattan metric might work better considering streets are a thing...
X = df_new[['Latitude','Longitude']]
kmeans2 = KMeans(n_clusters = 2)
kmeans3 = KMeans(n_clusters = 3)
kmeans4 = KMeans(n_clusters = 4)
kmeans5 = KMeans(n_clusters = 5)

Result2 = kmeans2.fit_predict(X)
Result3 = kmeans3.fit_predict(X)
Result4 = kmeans4.fit_predict(X)
Result5 = kmeans5.fit_predict(X)

df_new['Cluster Labels K = 2'] = Result2
df_new['Cluster Labels K = 3'] = Result3
df_new['Cluster Labels K = 4'] = Result4
df_new['Cluster Labels K = 5'] = Result5
df_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels 1,Cluster Labels K = 2,Cluster Labels K = 3,Cluster Labels K = 4,Cluster Labels K = 5
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,0,0,0,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,0,0,2,4
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,1,1,3,1
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,1,2,1,2
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1,1,2,1,3
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,1,1,3,1
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0,0,1,3,1
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,1,1,2,1,3


In [87]:
#Visualizing the plots for K =2
m2 = folium.Map(location=[43.6532, -79.3832], zoom_start= 12)

for i in range(len(df_new.index)):
    if df_new.iloc[i,6] == 0:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='red')).add_to(m2)
    if df_new.iloc[i,6] == 1:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='blue')).add_to(m2)
    

In [88]:
m2

In [91]:
#Visualizing the plots for K = 3
m3 = folium.Map(location=[43.6532, -79.3832], zoom_start= 12)

for i in range(len(df_new.index)):
    if df_new.iloc[i,7] == 0:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='red')).add_to(m3)
    if df_new.iloc[i,7] == 1:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='blue')).add_to(m3)
    if df_new.iloc[i,7] == 2:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='green')).add_to(m3)

In [92]:
m3

In [100]:
#Visualizing the plots for K = 4
m4 = folium.Map(location=[43.6532, -79.3832], zoom_start= 12)

for i in range(len(df_new.index)):
    if df_new.iloc[i,8] == 0:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='red')).add_to(m4)
    if df_new.iloc[i,8] == 1:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='blue')).add_to(m4)
    if df_new.iloc[i,8] == 2:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='green')).add_to(m4)
    if df_new.iloc[i,8] == 3:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='orange')).add_to(m4)

In [101]:
m4

In [104]:
#Visualizing the plots for K = 5
m5 = folium.Map(location=[43.6532, -79.3832], zoom_start= 12)

for i in range(len(df_new.index)):
    if df_new.iloc[i,9] == 0:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='red')).add_to(m5)
    if df_new.iloc[i,9] == 1:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='blue')).add_to(m5)
    if df_new.iloc[i,9] == 2:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='green')).add_to(m5)
    if df_new.iloc[i,9] == 3:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='orange')).add_to(m5)
    if df_new.iloc[i,9] == 4:
        folium.Marker([df_new.iloc[i,3], df_new.iloc[i,4]],icon=folium.Icon(color='purple')).add_to(m5)
        
m5